In [1]:
# 95% CI from Monte Carlo
import pandas as pd
import numpy as np
pd.set_option('display.float_format', '{:.2e}'.format)

# Data import
file_path = "/Users/elchulito/Library/CloudStorage/OneDrive-polymtl/PlasticFADE.xlsx"  # CHECK: confirm file path
sheet_name = "Uncertainty"
data_CI = pd.read_excel(file_path, sheet_name=sheet_name, usecols="A:D", skiprows=1)
data_CI = data_CI.iloc[30:36] # Row index minus 3, change this range for other polymers
print(data_CI)

# Parameter estimates
x_i, tau_i, y_i, theta_i, z_i, eta_i = data_CI.iloc[:, 2].values
# Standard deviations of parameters
x_i_std, tau_i_std, y_i_std, theta_i_std, z_i_std, eta_i_std = data_CI.iloc[:, 3].values

           Process Parameter  Estimate Standard deviation
30  PE degradation       x_i  4.72e-09           2.48e-09
31             NaN     tau_i  1.35e+00           1.93e-10
32             NaN       y_i  3.68e-04           1.92e-06
33             NaN   theta_i  4.99e+00           1.14e-09
34             NaN       z_i  5.33e-05           1.33e-05
35             NaN     eta_i  6.51e-01           1.46e-08


In [3]:
# Input parameters
data_input = pd.read_excel(file_path, sheet_name="Results", usecols="A:D,F", skiprows=13)
data_input = data_input[data_input.iloc[:, 0] == "PE"]  # Change for different polymer types
print(data_input)
data_input.columns = ['Polymer', 'Compartment', 's', 'I_j', 'C_j']

# Monte Carlo setup
N = 10000
np.random.seed(42)
results = []

# Loop through each row of input
for index, row in data_input.iterrows():
    s = row['s']
    I_j = row['I_j']
    C_j = row['C_j']

    # Monte Carlo with log-normal distribution
    N = 10000
    x_i_samples = np.random.lognormal(np.log(x_i), x_i_std, N)
    tau_i_samples = np.random.lognormal(np.log(tau_i), tau_i_std, N)
    y_i_samples = np.random.lognormal(np.log(y_i), y_i_std, N)
    theta_i_samples = np.random.lognormal(np.log(theta_i), theta_i_std, N)
    z_i_samples = np.random.lognormal(np.log(z_i), z_i_std, N)
    eta_i_samples = np.random.lognormal(np.log(eta_i), eta_i_std, N)

    # Compute k_degr for each sample
    k_samples = x_i_samples * (s**tau_i_samples) * (y_i_samples * I_j**theta_i_samples + z_i_samples * C_j**eta_i_samples)
    k_samples = k_samples[np.isfinite(k_samples)]  # Filter invalid samples (good habit, especially when NaNs are found in the CIs)

    # 95% CI in log-space
    log_k = np.log10(k_samples)
    lower_bound = 10 ** np.percentile(log_k, 2.5)
    upper_bound = 10 ** np.percentile(log_k, 97.5)
    k_point = x_i * (s**tau_i) * (y_i * I_j**theta_i + z_i * C_j**eta_i)
    
    results.append({'Compartment': row['Compartment'], 'k_point': k_point, 'CI_lower': lower_bound, 'CI_upper': upper_bound})

# Display results
results_CI = pd.DataFrame(results)
print("\n", results_CI)
print(f"\n{N - len(k_samples)} out of {N} samples were invalid and removed.")

   Polymer (i) Compartment (j)  SA:V [cm-1]  I_j [W/m2]  C_j [CFU/ml]
42          PE             Air           25    1.00e+01      5.00e-01
43          PE         Topsoil           25    1.00e-01      6.70e+08
44          PE         Subsoil           25    0.00e+00      1.21e+08
45          PE           Beach           25    1.25e+01      1.25e+07
46          PE   Water surface           25    1.00e+01      2.50e+05
47          PE    Water column           25    0.00e+00      3.85e+04
48          PE        Sediment           25    0.00e+00      4.82e+05

      Compartment  k_point  CI_lower  CI_upper
0            Air 1.30e-05  1.30e-05  1.30e-05
1        Topsoil 1.07e-05  1.07e-05  1.07e-05
2        Subsoil 3.51e-06  3.51e-06  3.51e-06
3          Beach 4.03e-05  4.03e-05  4.03e-05
4  Water surface 1.30e-05  1.30e-05  1.30e-05
5   Water column 1.86e-08  1.86e-08  1.86e-08
6       Sediment 9.63e-08  9.63e-08  9.63e-08

0 out of 10000 samples were invalid and removed.


In [5]:
# Write confidence intervals back to Excel (without modifying)
import xlwings as xw

wb = xw.Book(file_path)
sheet = wb.sheets["Results"]

start_row = 57  # CHECK: Change this index for other polymers
sheet.range(f'M{start_row}').options(index=False, header=False).value = results_CI['CI_lower'].values.reshape(-1, 1)
sheet.range(f'N{start_row}').options(index=False, header=False).value = results_CI['CI_upper'].values.reshape(-1, 1)

wb.save()
wb.close()